In [1]:
from bs4 import BeautifulSoup
import requests
import os
import wget
import time
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\Time series - Annex I"
import urllib.request
import logging
from selenium.webdriver.support.ui import Select
import datetime
from Hashing.HashScrapedData import _hashing
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time
import os
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI
chrome_options = webdriver.ChromeOptions()
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\Time series - Annex I" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
chrome_options.add_experimental_option('prefs', prefs)
# chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()
driver.get('https://di.unfccc.int/time_series')
time.sleep(6)
PageContent = driver.find_element(By.CLASS_NAME,'PageContent')
theme = PageContent.find_element(By.TAG_NAME,'h1').text
tag = theme.split(' -')[0]
select_element = PageContent.find_element(By.CLASS_NAME,'party-data-filter')
select = Select(select_element)
option_list = select.options
for op in option_list:
    select.select_by_visible_text(op.text)
    time.sleep(3)
    topic = PageContent.find_element(By.TAG_NAME,'p').text.replace('/',', ').replace(':','')
    title = tag + ' ' + topic
    table = driver.find_element(By.CLASS_NAME,'dataTable.no-footer')
    head = []
    thead = table.find_element(By.TAG_NAME,'thead').find_elements(By.TAG_NAME,'th')
    for th in thead:
        head.append(th.text)
    rows = table.find_elements(By.TAG_NAME,'tr')
    l = []
    for tr in rows[1:]:
        cells = tr.find_elements(By.TAG_NAME,'td')
        cells_text = [cell.text for cell in cells]
        l.append(cells_text)
    df = pd.DataFrame(l,columns=head) 
    df.to_excel(f'{base_path}\\{title}.xlsx',index = False, encoding="utf-8")
    notes = driver.find_element(By.CLASS_NAME,'disclaimer-notes').text
    print(notes)
    PageContent = driver.find_element(By.CLASS_NAME,'PageContent')
    theme = PageContent.find_element(By.TAG_NAME,'h1').text
    tag = theme.split(' -')[0]
    select_element = PageContent.find_element(By.CLASS_NAME,'party-data-filter')
    select = Select(select_element)
    option_list = select.options
    try:
        BodyDict = {
        "JobPath":f'{base_path}\\{title}.xlsx', #* Point to downloaded data for conversion #
        "JsonDetails":{
                ## Required
                "organisation": "UN-agencies",
                "source": "UNFCCC",
                "source_description" : "The United Nations Framework Convention on Climate Change established an international environmental treaty to combat 'dangerous human interference with the climate system', in part by stabilizing greenhouse gas concentrations in the atmosphere.",
                "source_url" : "https://viewpoint.eiu.com/data",
                "table" : title ,
                "description" : notes  ,
                ## Optional
                "JobType": "JSON",
                "CleanPush": True,
                "Server": "str",
                "UseJsonFormatForSQL":  False,
                "CleanReplace":True,
                "MergeSchema": False,
                "tags": [
                            {"name": 'time series'}
                        ],
                "additional_data_sources": [{       
                        "name": '',        
                        "url": ''  ## this object will be ignored if "name" is empty    }
                }],
                "limitations":'',
                "concept":  '',
                "periodicity":  '',
                "topic": title ,
                "created": '',                       #* this should follow the following formats %Y-%m-%dT%H:%M:%S" or "%Y-%m-%d"
                "last_modified":'' ,                #* ""               ""                  ""              ""
                "TriggerTalend" :  False,    #* initialise to True for production
                "SavePathForJsonOutput": "//10.30.31.77/data_collection_dump/TestData" #* initialise as empty string for production.
            }
        }
        tablenom = BodyDict['JsonDetails']['table']

        hashmessage = _hashing(BodyDict['JsonDetails']['source'], tablenom, BodyDict["JobPath"])

        if hashmessage["Trigger_InferSchema"] == True and hashmessage["Success"] == True:

            TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
            TriggerInferShemaToJsonAPIClass.TriggerAPI()
            logging.info(f"Conversion successful - {tablenom}, hashmessage: {hashmessage['message']}")

        # logging.info(f"Conversion successful for {dataset}")
        elif hashmessage['Success'] == True and hashmessage['Trigger_InferSchema'] == False:

            # dont trigger conversion nor talend
            logging.info(f"{hashmessage['message']}")

        elif hashmessage['Success'] == False:
            logging.info(f"Hashing error or Unexpected Issue: {hashmessage['message']}")
    except  Exception as err:
        print(err)

In [39]:
# logging.basicConfig(handlers=[logging.FileHandler("//10.30.31.77/data_collection_dump/TACFolder/ScraperLogs/IRENA/IRENA.log"),
#                                 logging.StreamHandler()], level=logging.INFO)
chrome_options = webdriver.ChromeOptions()
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\Time series - Annex I" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
chrome_options.add_experimental_option('prefs', prefs)
# chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome 114.0.5735
INFO:WDM:Driver [C:\Users\10235555\.wdm\drivers\chromedriver\win32\114.0.5735\chromedriver.exe] found in cache


In [43]:
# time series
driver.get('https://di.unfccc.int/time_series')
time.sleep(6)
PageContent = driver.find_element(By.CLASS_NAME,'PageContent')
theme = PageContent.find_element(By.TAG_NAME,'h1').text
tag = theme.split(' -')[0]
select_element = PageContent.find_element(By.CLASS_NAME,'party-data-filter')
select = Select(select_element)
option_list = select.options
for op in option_list:
    select.select_by_visible_text(op.text)
    time.sleep(3)
    topic = PageContent.find_element(By.TAG_NAME,'p').text.replace('/',', ').replace(':','')
    title = tag + ' ' + topic
    table = driver.find_element(By.CLASS_NAME,'dataTable.no-footer')
    head = []
    thead = table.find_element(By.TAG_NAME,'thead').find_elements(By.TAG_NAME,'th')
    for th in thead:
        head.append(th.text)
    rows = table.find_elements(By.TAG_NAME,'tr')
    l = []
    for tr in rows[1:]:
        cells = tr.find_elements(By.TAG_NAME,'td')
        cells_text = [cell.text for cell in cells]
        l.append(cells_text)
    df = pd.DataFrame(l,columns=head) 
    df.to_excel(f'{base_path}\\{title}.xlsx',index = False, encoding="utf-8")
    notes = driver.find_element(By.CLASS_NAME,'disclaimer-notes').text
    print(notes)
    PageContent = driver.find_element(By.CLASS_NAME,'PageContent')
    theme = PageContent.find_element(By.TAG_NAME,'h1').text
    tag = theme.split(' -')[0]
    select_element = PageContent.find_element(By.CLASS_NAME,'party-data-filter')
    select = Select(select_element)
    option_list = select.options


Note 1: The reporting and review requirements for GHG inventories are different for Annex I and non-Annex I Parties. The definition format of data for emissions/removals from the forestry sector is different for Annex I and non-Annex I Parties.
Note 2: Base year data in the data interface relate to the base year under the Climate Change Convention (UNFCCC). The base year under the Convention is defined slightly different than the base year under the Kyoto Protocol. An exception is made for European Union (KP) whereby the base year under the Kyoto Protocol is displayed.
Note 3: – means "No data available"
Note 4: Data displayed on the data interface are "as received" from Parties. The publication of Party submissions on this website does not imply the expression of any opinion whatsoever on the part of the UNFCCC or the Secretariat of the United Nations concerning the legal status of any country, territory, city or area or of its authorities, or concerning the delimitation of its fronti

In [16]:
import os 
path_ckan = ''
ckan_f = []
path_data = ''
data_f = []
for file in os.listdir(path_data):
    data = file.split['.xlsx'][0]
    data_f.append(data)
for ck in os.listdir(path_ckan):
    ck_data = ck.split('_')[0]
    ckan_f.append(ckan_f)
for c in ckan_f:
    if c not in data_f:
        print(c)

FileNotFoundError: [WinError 3] The system cannot find the path specified: ''

In [66]:
path_ckan = r'\\10.30.31.77\data_collection_dump\TestData\UNFCCC\Global map\unfccc_Data'
uniq = []
dup = []
import os
for file in os.listdir(path_ckan):
        if file.startswith('CKAN'):
            file = file.split('_')[1]
            if file not in uniq :  
                uniq.append(file)
            else:
                dup.append(file)

In [67]:
dup

['All Annex I Parties Other Aggregate F gases in kt CO2 eq',
 'All Annex I Parties Other PFCs in kt CO2 eq',
 'All Annex I Parties Other SF6 in kt CO2 eq',
 'All Annex I Parties Other Unspecified mix of HFCs and PFCs in kt CO2 eq',
 'All Annex I Parties Other CO2 in kt CO2 eq',
 'All Annex I Parties Other N2O in kt CO2 eq',
 'All Annex I Parties Other HFCs in kt CO2 eq',
 'All Annex I Parties Other NF3 in kt CO2 eq',
 'All Annex I Parties Other CH4 in kt CO2 eq',
 'All Annex I Parties Other Aggregate GHGs in kt CO2 eq']

In [ ]:
try:
    BodyDict = {
    "JobPath":f'//10.30.31.77/data_collection_dump/RawData/UNFCCC/Time Series/{title}.xlsx', #* Point to downloaded data for conversion #
    "JsonDetails":{
            ## Required
            "organisation": "un-agencies",
            "source": "Test UNFCCC",
            "source_description" : "The United Nations Framework Convention on Climate Change established an international environmental treaty to combat 'dangerous human interference with the climate system', in part by stabilizing greenhouse gas concentrations in the atmosphere.",
            "source_url" : "https://di.unfccc.int",
            "table" : title ,
            "description" : notes  ,
            ## Optional
            "JobType": "JSON",
            "CleanPush": True,
            "Server": "str",
            "UseJsonFormatForSQL":  False,
            "CleanReplace":True,
            "MergeSchema": False,
            "tags": [
                        {"name": tag}
                    ],
            "additional_data_sources": [{       
                    "name": '',        
                    "url": ''  ## this object will be ignored if "name" is empty    }
            }],
            "limitations":'',
            "concept":  '',
            "periodicity":  '',
            "topic": title ,
            "created": '',                       #* this should follow the following formats %Y-%m-%dT%H:%M:%S" or "%Y-%m-%d"
            "last_modified":'' ,                #* ""               ""                  ""              ""
            "TriggerTalend" :  False,    #* initialise to True for production
            "SavePathForJsonOutput": "//10.30.31.77/data_collection_dump/TestData/UNFCCC Test/Time Series" #* initialise as empty string for production.
        }
    }
    # tablenom = BodyDict['JsonDetails']['table']

    # hashmessage = _hashing(BodyDict['JsonDetails']['source'], tablenom, BodyDict["JobPath"])

    # if hashmessage["Trigger_InferSchema"] == True and hashmessage["Success"] == True:

    TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
    TriggerInferShemaToJsonAPIClass.TriggerAPI()
    logging.info(f"Conversion successful - {title} ")
    print(BodyDict)
    #     logging.info(f"Conversion successful - {tablenom}, hashmessage: {hashmessage['message']}")

    # # logging.info(f"Conversion successful for {dataset}")
    # elif hashmessage['Success'] == True and hashmessage['Trigger_InferSchema'] == False:

    #     # dont trigger conversion nor talend
    #     logging.info(f"{hashmessage['message']}")

    # elif hashmessage['Success'] == False:
    #     logging.info(f"Hashing error or Unexpected Issue: {hashmessage['message']}")
except  Exception as err:
    print(err)